In [ ]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [ ]:
soup = BeautifulSoup(open('D:\\Drive\\VSCode\\Python\\messages.html', encoding='utf-8'), 'html.parser')
tags = soup.findAll('div', 'body')

In [ ]:
strs = [str(tag) for tag in tags]
stat_strs = [s for s in strs if 'За последние сутки в России' in s]

In [ ]:
data = dict()
for message in stat_strs:
    date = re.search(r'title=\"(.*?) ', message).group(1)
    daily = re.search(r'🔸(.*?)Всего на сегодняшний день в России', message).group(1)
    data.update({date : daily})

regex_for_tag = re.compile('<.*?>')
for date in data:
    data[date] = data[date].split('🔸')

for date in data:
    for i in range(len(data[date])):
        data[date][i] = re.sub(regex_for_tag, '', data[date][i])
        data[date][i] = re.sub('\xa0 ', '', data[date][i])
        data[date][i] = data[date][i].strip()
        try:
            spl = data[date][i].split(' - ')
            data[date][i] = {spl[0] : int(spl[1].replace(' ', ''))}
        except:
            spl = data[date][i].split('- ')
            data[date][i] = {spl[0] : int(spl[1])}

    data[date].insert(0, {'Дата' : date})

    temp_d = {}
    for i in range(len(data[date])):
        temp_d.update(data[date][i])
    data[date] = temp_d


data = list(data.values())

In [ ]:
df = pd.DataFrame(data)
display(df)

In [ ]:
li = list(df.columns)
li.sort()
print(li)

In [ ]:
def join_col(df, cols):
    if len(cols) == 2:
        a = cols[0]
        b = cols[1]
        df[a] = df[a].fillna(df[b])
        df = df.drop(columns = [b])
    if len(cols) == 3:
        a = cols[0]
        b = cols[1]
        c = cols[2]
        df[b] = df[b].fillna(df[c])
        df[a] = df[a].fillna(df[b])
        df = df.drop(columns = [b, c])
    return df

In [ ]:
df = join_col(df, ['Еврейская АО', 'Еврейская автономная область'])
df = join_col(df, ['Кировская область', 'Кировская область '])
df = join_col(df, ['Ненецкий АО', 'Ненецкий  АО', 'Ненецкий автономный округ'])
df = join_col(df, ['Орловская область', 'Орловская область '])
df = join_col(df, ['Республика Дагестан', 'Республика Дагестан '])
df = join_col(df, ['Республика Северная Осетия', 'Республика Северная Осетия-Алания'])
df = join_col(df, ['Санкт-Петербург', 'Санкт-Петербург  '])
df = join_col(df, ['Севастополь', 'г. Севастополь', 'г.Севастополь'])
df = join_col(df, ['Ханты-Мансийский АО', 'Ханты-Мансийский автономный округ'])
df = join_col(df, ['Чукотский АО', 'Чукотский автономный округ'])
df = join_col(df, ['Ямало-Ненецкий АО', 'Ямало-Ненецкий автономный округ'])
df = join_col(df, ['Чеченская Республика', 'Чеченская республика'])
df = join_col(df, ['Республика Алтай', 'Республика  Алтай'])
df = join_col(df, ['Чувашская Республика', 'Республика Чувашия'])

display(df)

In [ ]:
df = df.fillna(0)
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y', errors='ignore')
cols = df.columns.drop('Дата')
display(df)

In [ ]:
df_pop = pd.read_csv('population.csv', encoding='utf-8', header=None)
display(df_pop)

In [ ]:
df_pop = df_pop.T
display(df_pop)

In [ ]:
new_header = df_pop.iloc[0]
df_pop = df_pop.iloc[[1]]
df_pop.columns = new_header
df_pop = df_pop.reset_index()
df_pop = df_pop.drop(columns='index')
display(df_pop)

In [ ]:
df1 = pd.DataFrame()
regions = list(df_pop.columns)
for region in regions:
    df1[region] = (df[region] * 1000000) / df_pop.at[0, region]

df1.insert(loc=0, column='Дата', value=df['Дата'])

display(df)

In [ ]:
display(df1)

In [ ]:
zipfile = "zip:///Drive//VSCode//Python//ne_10m_admin_1_states_provinces.zip"
gdf = gpd.read_file(zipfile, encoding = 'UTF-8')
display(gdf)

In [ ]:
flag = gdf['gu_a3'] == 'RUS'
ru_gdf = gdf[flag]
ru_gdf = ru_gdf[['name', 'name_local', 'geometry']]
ru_gdf = ru_gdf.reset_index()
display(ru_gdf)

In [ ]:
ru_gdf.at[5, 'name_local'] = 'Республика Северная Осетия'
ru_gdf.at[6, 'name_local'] = 'Республика Ингушетия'
ru_gdf.at[7, 'name_local'] = 'Чеченская Республика'
ru_gdf.at[23, 'name_local'] = 'Еврейская АО'
ru_gdf.at[3, 'name_local'] = 'Карачаево-Черкесская Республика'
ru_gdf.at[36, 'name_local'] = 'Республика Крым'
ru_gdf.at[39, 'name_local'] = 'Севастополь'
ru_gdf.at[40, 'name_local'] = 'Чукотский АО'
ru_gdf.at[41, 'name_local'] = 'Ямало-Ненецкий АО'
ru_gdf.at[42, 'name_local'] = 'Ненецкий АО'
ru_gdf.at[43, 'name_local'] = 'Республика Саха (Якутия)'
ru_gdf.at[44, 'name_local'] = 'Санкт-Петербург'
ru_gdf.at[45, 'name_local'] = 'Архангельская область'
ru_gdf.at[48, 'name_local'] = 'Камчатский край'
ru_gdf.at[52, 'name_local'] = 'Ханты-Мансийский АО'
ru_gdf.at[76, 'name_local'] = 'Московская область'
ru_gdf.at[77, 'name_local'] = 'Москва'
ru_gdf.at[82, 'name_local'] = 'Пермский край'
ru_gdf = ru_gdf.sort_values(by='name_local')
ru_gdf = ru_gdf.to_crs({'init' :'epsg:3576'})
ru_gdf = ru_gdf.reset_index()
ru_gdf = ru_gdf.drop(index=80)
ru_gdf = ru_gdf.drop(index=85)
display(ru_gdf)

In [ ]:
data2020_12_27 = df1.loc[275]
data2020_12_27 = pd.DataFrame(data2020_12_27)
data2020_12_27 = data2020_12_27.drop(index='Дата')
data2020_12_27.columns = ['n']
data2020_12_27 = data2020_12_27.sort_index() 
data2020_12_27['name'] = data2020_12_27.index
data2020_12_27 = data2020_12_27.reset_index()
data2020_12_27 = data2020_12_27.drop(columns='name')


display(data2020_12_27)

In [ ]:
gdf2020_12_27 = ru_gdf.merge(data2020_12_27, right_on='index', left_on='name_local')
gdf2020_12_27 = gdf2020_12_27.astype({'n': 'float'})
display(gdf2020_12_27)

In [ ]:
fig, ax = plt.subplots(1, 1)
hmap = gdf2020_12_27.plot(column='n',ax=ax,legend=True,legend_kwds={'label': "Заболевшие за 27.12.2020 на 1 млн человек", 'orientation': "horizontal"})
hmap.figure.savefig('27.12.png', dpi=500, transparent=False)

In [ ]:
def plotter(i):
    data = df1.loc[i]
    date = data.iloc[0]
    date = str(date)[:10]
    data = pd.DataFrame(data)
    data = data.drop(index='Дата')
    data.columns = ['n']
    data = data.sort_index() 
    data['name'] = data.index
    data = data.reset_index()
    data = data.drop(columns='name')
    gdf = ru_gdf.merge(data, right_on='index', left_on='name_local')
    gdf = gdf.astype({'n': 'float'})

    plt.rcParams["figure.facecolor"] = 'w'
    fig, ax = plt.subplots(1, 1)
    plt.axis('off')
    hmap = gdf.plot(column='n',ax=ax,legend=True,legend_kwds={'label': "Заболевшие за " + date + " на 1 млн человек", 'orientation': "vertical"}, vmin=0, vmax=700, cmap='rainbow')
    hmap.figure.savefig('hmaps/' + str(i) + '.png', dpi=200, transparent=False)


In [ ]:
#for i in range(276):
    #plotter(i)